In [1]:
%pip install ctgan

   ---------------------------------------- 0.0/63.8 kB ? eta -:--:--
   ---------------------------------------- 63.8/63.8 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.8 MB 5.6 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.8 MB 5.3 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.8 MB 6.0 MB/s eta 0:00:01
   ------------------------ --------------- 1.1/1.8 MB 6.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.8 MB 5.8 MB/s eta 0:00:01
   ------------------------------- -------- 1.4/1.8 MB 5.2 MB/s eta 0:00:01
   --------------------------------- ------ 1.5/1.8 MB 5.0 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.8 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.8 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 4.0 MB/s eta 0:00:00
Note: you may need to r

In [2]:
from ctgan import CTGAN
import pandas as pd

In [19]:
real_data = pd.read_csv('../Dataset/Aggregated_Sleep.csv')
real_data.head()
real_data.drop(['patient_id', 'window_start', 'SE'], axis=1, inplace=True)
real_data.dropna()
real_data.head()

,mean_HR,HR_var,mean_RR,RR_var,WASO,SOL,TIB,TST,snoring_counts,agitation
0,65.039514,3.345222,11.917933,1.348875,270.0,10.0,540.0,204.0,0.0,0.0
1,67.176030,2.739798,11.456929,0.992935,124.0,19.0,349.0,179.0,0.0,0.0
2,64.623583,3.076862,11.977324,1.061488,222.0,15.0,569.0,238.0,0.0,0.0
3,62.416667,4.372302,11.397849,1.110011,61.0,13.0,401.0,309.0,80.0,0.0
4,68.082569,2.632167,13.330275,1.171064,77.0,12.0,149.0,49.0,0.0,0.0


In [20]:
#only keep the rows that have Agitation=1
real_data = real_data[real_data['agitation']==1]
real_data.head()
#check data type of each column
real_data.dtypes
#change snoring_counts and agitation to int
real_data['snoring_counts'] = real_data['snoring_counts'].astype(int)
real_data['agitation'] = real_data['agitation'].astype(int)
#make WASO, TST, TIB, SE to int
real_data['WASO'] = real_data['WASO'].astype(int)
real_data['TST'] = real_data['TST'].astype(int)
real_data['TIB'] = real_data['TIB'].astype(int)
real_data['SOL'] = real_data['SOL'].astype(int)
real_data.dtypes

mean_HR           float64
HR_var            float64
mean_RR           float64
RR_var            float64
WASO                int32
SOL                 int32
TIB                 int32
TST                 int32
snoring_counts      int32
agitation           int32
dtype: object

In [21]:
real_data

,mean_HR,HR_var,mean_RR,RR_var,WASO,SOL,TIB,TST,snoring_counts,agitation
10,62.223577,9.519626,14.321138,2.350536,246,15,387,108,0,1
18,70.110837,6.473881,14.088670,2.058891,26,17,405,327,3,1
19,63.283757,11.216572,15.438356,2.714452,80,8,529,336,4,1
23,69.777414,6.212590,16.291326,2.285429,201,94,610,288,0,1
45,71.997938,7.793963,15.896907,2.744960,142,100,484,193,0,1
48,67.722222,6.350536,16.794444,2.432824,0,72,359,79,0,1
49,68.510730,5.106060,16.394850,2.517542,392,0,953,493,0,1
52,67.797633,5.795937,15.875740,2.515345,407,241,859,151,0,1
316,57.208547,3.604259,13.663248,1.312910,29,19,595,470,27,1
321,58.400697,4.784326,14.398955,1.637070,206,6,683,466,35,1


In [22]:
# Names of the columns that are discrete
discrete_columns = [
    'snoring_counts',
    'agitation',
    'TST',
    'TIB',
    'SOL',
    'WASO'
]

ctgan = CTGAN(epochs=50)
ctgan.fit(real_data, discrete_columns)

# Create synthetic data
synthetic_data = ctgan.sample(50)
synthetic_data

,mean_HR,HR_var,mean_RR,RR_var,WASO,SOL,TIB,TST,snoring_counts,agitation
0,72.100599,1.377871,11.218227,1.767884,29,8,595,466,0,1
1,73.286386,3.306449,12.259952,1.990000,80,6,359,344,0,1
2,52.167123,3.828592,12.369813,1.643849,55,19,553,327,46,1
3,66.703701,4.708587,13.838701,2.131371,48,19,529,475,3,1
4,56.561923,5.257835,13.476077,3.437994,0,11,581,483,98,1
5,74.954084,5.973744,11.025714,2.352615,40,15,595,496,27,1
6,49.922053,6.918884,13.548420,2.075675,392,18,581,392,0,1
7,51.623135,8.183268,11.671924,1.905402,16,26,953,336,27,1
8,54.081997,4.658750,14.065808,1.307132,33,19,496,419,3,1
9,55.070447,0.772821,11.875361,2.555092,40,14,595,493,0,1


In [23]:
#create a new column called SE which is placed between TST and snoring_counts, which is TST/TIB for each row
synthetic_data['SE'] = (synthetic_data['TST'] / synthetic_data['TIB']) * 100
synthetic_data

,mean_HR,HR_var,mean_RR,RR_var,WASO,SOL,TIB,TST,snoring_counts,agitation,SE
0,72.100599,1.377871,11.218227,1.767884,29,8,595,466,0,1,78.319328
1,73.286386,3.306449,12.259952,1.990000,80,6,359,344,0,1,95.821727
2,52.167123,3.828592,12.369813,1.643849,55,19,553,327,46,1,59.132007
3,66.703701,4.708587,13.838701,2.131371,48,19,529,475,3,1,89.792060
4,56.561923,5.257835,13.476077,3.437994,0,11,581,483,98,1,83.132530
5,74.954084,5.973744,11.025714,2.352615,40,15,595,496,27,1,83.361345
6,49.922053,6.918884,13.548420,2.075675,392,18,581,392,0,1,67.469880
7,51.623135,8.183268,11.671924,1.905402,16,26,953,336,27,1,35.257083
8,54.081997,4.658750,14.065808,1.307132,33,19,496,419,3,1,84.475806
9,55.070447,0.772821,11.875361,2.555092,40,14,595,493,0,1,82.857143


@inproceedings{ctgan,
  title={Modeling Tabular data using Conditional GAN},
  author={Xu, Lei and Skoularidou, Maria and Cuesta-Infante, Alfredo and Veeramachaneni, Kalyan},
  booktitle={Advances in Neural Information Processing Systems},
  year={2019}
}

In [24]:
cols = list(synthetic_data.columns)
cols

['mean_HR',
 'HR_var',
 'mean_RR',
 'RR_var',
 'WASO',
 'SOL',
 'TIB',
 'TST',
 'snoring_counts',
 'agitation',
 'SE']

In [25]:
# Pop the ninth index column 'J'
moving_col = cols.pop(10)  # Adjust index for 0-based Python indexing (9 is the 10th position)
# Insert it before the 7th index ('H')
cols.insert(8, moving_col)
# Re-assign the columns attribute
synthetic_data = synthetic_data[cols]
synthetic_data

,mean_HR,HR_var,mean_RR,RR_var,WASO,SOL,TIB,TST,SE,snoring_counts,agitation
0,72.100599,1.377871,11.218227,1.767884,29,8,595,466,78.319328,0,1
1,73.286386,3.306449,12.259952,1.990000,80,6,359,344,95.821727,0,1
2,52.167123,3.828592,12.369813,1.643849,55,19,553,327,59.132007,46,1
3,66.703701,4.708587,13.838701,2.131371,48,19,529,475,89.792060,3,1
4,56.561923,5.257835,13.476077,3.437994,0,11,581,483,83.132530,98,1
5,74.954084,5.973744,11.025714,2.352615,40,15,595,496,83.361345,27,1
6,49.922053,6.918884,13.548420,2.075675,392,18,581,392,67.469880,0,1
7,51.623135,8.183268,11.671924,1.905402,16,26,953,336,35.257083,27,1
8,54.081997,4.658750,14.065808,1.307132,33,19,496,419,84.475806,3,1
9,55.070447,0.772821,11.875361,2.555092,40,14,595,493,82.857143,0,1


In [26]:
#drop rows of SE where the value is greater than 100
synthetic_data = synthetic_data[synthetic_data['SE'] <= 100]
synthetic_data

,mean_HR,HR_var,mean_RR,RR_var,WASO,SOL,TIB,TST,SE,snoring_counts,agitation
0,72.100599,1.377871,11.218227,1.767884,29,8,595,466,78.319328,0,1
1,73.286386,3.306449,12.259952,1.990000,80,6,359,344,95.821727,0,1
2,52.167123,3.828592,12.369813,1.643849,55,19,553,327,59.132007,46,1
3,66.703701,4.708587,13.838701,2.131371,48,19,529,475,89.792060,3,1
4,56.561923,5.257835,13.476077,3.437994,0,11,581,483,83.132530,98,1
5,74.954084,5.973744,11.025714,2.352615,40,15,595,496,83.361345,27,1
6,49.922053,6.918884,13.548420,2.075675,392,18,581,392,67.469880,0,1
7,51.623135,8.183268,11.671924,1.905402,16,26,953,336,35.257083,27,1
8,54.081997,4.658750,14.065808,1.307132,33,19,496,419,84.475806,3,1
9,55.070447,0.772821,11.875361,2.555092,40,14,595,493,82.857143,0,1


In [27]:
synthetic_data.shape

(43, 11)